**<font color=black size=5>朴素贝叶斯</font>**

**1** 介绍朴素贝叶斯算法的基本原理

**2** 手动实现传统朴素贝叶斯模型MyCatogricalNB，并通过西瓜分类案例，对比分析本文算法与sklearn库的结果

**3** 手动实现高斯朴素贝叶斯模型MyGaussianNB，并通过鸢尾花分类案例，对比分析本文算法与sklearn库的结果

In [1]:
from sklearn.datasets import load_iris
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.preprocessing import OrdinalEncoder
from scipy.special import logsumexp
from collections import Counter
import numpy as np
import pandas as pd

**<font color=black size=4>1 算法概述</font>**

**1.1 提出问题**

给定已知分类标签的数据集$\mathop D \limits_{n \times (d+1)}$，输入一个未知分类标签的的特征向量$\mathop x \limits_{1 \times d} =(x^{(1)},x^{(2)},...,x^{(d)})$，问输出的分类标签$y$是什么？

**1.2 贝叶斯定理**

根据贝叶斯定理，上述问题可以转换为如下数学表达：
$$P(Y=c_k|X=x) = \frac{P(X=x|Y=c_k)P(Y=c_k)}{P(X=x)}\tag{1}$$
其中：$P(Y=c_k)$是先验概率；$P(X=x|Y=c_k)$是条件概率。需要求解的问题可以理解为求出在输入$x$的条件下，$y$是第$k$类的概率，其中的最大概率所对应的类别即为最终输出的分类标签。

**1.3 朴素贝叶斯**
朴素贝叶斯**假设特征之间条件独立**，则上述公式可以进一步变换为：

$$P(Y=c_k|X=x) = \frac{P(X=x|Y=c_k)P(Y=c_k)}{P(X=x)}=\frac{P(Y=c_k) \prod_{j=1}^{d}P(X^{(j)}=x^{(j)}|Y=c_k)}{P(X=x)}\tag{2}$$
其中：$x^{(j)}$表示$x$的第$j$个取值。

**1.4 模型求解**

由式(2)可知，对于一个$x$来说$P(x)$是定值，因此要**求最大概率，只需要比较分子的大小**，因此可以进一步简化问题为：求出$max[P(Y=c_k) \prod_{j=1}^{d}P(X^{(j)}=x^{(j)}|Y=c_k)]$，其对应的类别$c_k$即为最终$x$的分类标签。

那么问题是：如何计算$P(Y=c_k)$和$P(X^{(j)}=x^{(j)}|Y=c_k)$？

**（1）类别型特征**

（1a）极大似然估计

$$P(Y=c_k)=\frac{\sum_{i=1}^N I(y_i=c_k)}{N}\tag{3}$$

$$P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^{N}I(x_i^{(j)}=a_{jl},y_i=c_k)}{\sum_{i=1}^{N}I(y_i=c_k)}\tag{4}$$

其中：$I$为指示函数，满足括号内条件则返回1，反之则返回0；$a_{jl}$为所有样本中第$j$个特征的第$l$个取值，$x^{(j)}$可能取值的集合为$\{a_{j1},a_{j2},...,a_{jS_j}\}$；$x_i^{(j)}$为第$i$个样本的第$j$个特征的取值；$N$为样本总数。详细推导见附录。

（1b）贝叶斯估计

使用极大似然估计可能会出现所估计的概率值为0的情况，这会对计算后验概率造成影响。因而我们采用贝叶斯估计解决这

一问题。先验概率和条件概率的贝叶斯估计分别为

$$P_\lambda(Y=c_k)=\frac{\sum_{i=1}^N I(y_i=c_k)+\lambda}{N+K\lambda}\tag{5}$$

$$P_\lambda(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^{N}I(x_i^{(j)}=a_{jl},y_i=c_k)+\lambda}{\sum_{i=1}^{N}I(y_i=c_k)+S_j\lambda}\tag{6}$$

当$\lambda=1$时，称为拉普拉斯平滑。

**（2）连续型特征**

通常假设在$Y=c_k$的条件下，$x$服从某种分布（如高斯分布），利用概率密度函数来计算$P(X^{(j)}=x^{(j)}|Y=c_k)$，$P(Y=c_k)$的计算采用极大似然估计。

<font color=blue>更多关于朴素贝叶斯的知识见参考。</font>

**<font color=black size=4>2 传统朴素贝叶斯</font>**

**<font color=black size=3.5>2.1 手动实现</font>**

传统朴素贝叶斯一般处理的是类别型数据，sklearn中提供了[sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB)来处理此类问题。

本文手动实现传统朴素贝叶斯算法，部分实现细节参考sklearn中的相关源码。

In [2]:
class MyCategoricalNB(object):
    """传统朴素贝叶斯    
    Parameters
    ----------
    alpha: float 拉普拉斯平滑因子 默认值为1
    priors: ndarray of shape (n_classes,) 类别的先验概率，若不指定则根据数据自适应计算

    Attributes
    ----------
    class_prior_: ndarray of shape (n_classes,) 
        每个类别的概率
    class_log_prior_: ndarray of shape (n_classes,) 
        每个类别的概率(log)
    classes_: ndarray of shape (n_classes,) 
        已知数据集的分类标签
    class_count_: ndarray of shape (n_classes,) 
        每个类标签对应的样本个数
    n_features_: int 
        数据集中的特征个数
    n_categories_: ndarray of shape (n_features,) 
        每个特征可取值的个数
    category_count_: list of arrays of shape (n_features,) 
        其中每个feature对应的array的形状为(n_classes, n_categories),
        对应存储每个类别下该特征的每个取值的样本个数。
    feature_log_prob_: list of arrays of shape (n_features,)
        其中每个feature对应的array的形状为(n_classes, n_categories),
        对应存储每个类别下该特征的每个取值的条件概率。
    """

    def __init__(self, alpha=1.0, priors=None):
        self.alpha = alpha
        self.class_prior_ = priors
        self.class_log_prior_ = None
        self.classes_ = None
        self.class_count_ = None
        self.n_features_ = None
        self.n_categories_ = None
        self.category_count_ = []
        self.feature_log_prob_ = []

    def fit(self, X, y):
        """拟合传统朴素贝叶斯模型
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量
        y: ndarray of shape (n_samples,) 样本标签
        """
        c = dict(Counter(y))
        self.classes_ = np.array(list(c.keys()))
        self.class_count_ = np.array(list(c.values()))

        # 估计先验概率时不使用拉普拉斯平滑
        if self.class_prior_ is None:
            self.class_prior_ = self.class_count_ / X.shape[0]
        self.class_log_prior_ = np.log(self.class_prior_)

        self.n_features_ = X.shape[1]
        self.n_categories_ = np.array([len(np.unique(X[:, i]))
                                       for i in range(self.n_features_)])

        self.category_count_ = self._update_category_count(X, y)

        self.feature_log_prob_ = self._update_feature_log_prob(self.alpha)

    def predict(self, X):
        """预测分类标签
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples,) 分类标签
        """
        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_proba(self, X):
        """预测归一化概率
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples, n_classes) 归一化概率
        """
        return np.exp(self.predict_log_proba(X))

    def predict_log_proba(self, X):
        """预测归一化概率(log)
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples, n_classes) 归一化概率(log)
        """
        jll = self._joint_log_likelihood(X)

        # 归一化概率 e.g. log(pi / (p1+p2+...+pn)), i=1,2,...,n, n=n_classes
        log_prob_x = logsumexp(jll, axis=1)
        out = jll - np.atleast_2d(log_prob_x).T
        return out

    def _update_category_count(self, X, y):
        """更新每个类别下每个特征的每个取值的样本个数
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量
        y:  ndarray of shape (n_samples,) 样本标签

        Returns
        -------
        category_count_: list of arrays of shape (n_features,) 
            更新后的每个类别下每个特征的每个取值的样本个数
        """
        for i in range(self.n_features_):
            arr = np.empty(shape=(len(self.classes_), self.n_categories_[i]))
            valid_categories = np.unique(X[:, i])
            for j in range(len(self.classes_)):
                c_f = Counter(X[y == self.classes_[j]][:, i])
                for k in range(self.n_categories_[i]):
                    arr[j][k] = c_f[valid_categories[k]]
            self.category_count_.append(arr)
        return self.category_count_

    def _update_feature_log_prob(self, alpha):
        """更新特征取值的条件概率(log)
        Parameters
        ----------
        alpha: float 拉普拉斯平滑因子

        Returns
        -------
        feature_log_prob_: list of arrays of shape (n_features,) 
            更新后的特征取值的条件概率(log)
        """
        for i in range(self.n_features_):
            smoothed_cat_count = self.category_count_[i] + alpha
            smoothed_class_count = smoothed_cat_count.sum(axis=1)
            self.feature_log_prob_.append(np.log(smoothed_cat_count) -
                                          np.log(smoothed_class_count.reshape(-1, 1)))
        return self.feature_log_prob_

    def _joint_log_likelihood(self, X):
        """计算联合概率(log)
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        total_ll: ndarray of shape (n_samples, n_classes) 联合概率(log)
        """
        jll = np.zeros((X.shape[0], len(self.class_count_)))
        for i in range(self.n_features_):
            indices = X[:, i]
            jll += self.feature_log_prob_[i][:, indices].T
        total_ll = jll + self.class_log_prior_
        return total_ll

**<font color=black size=3.5>2.2 西瓜分类案例</font>**<br>

In [3]:
# 数据准备 来源于《机器学习》-周志华 西瓜数据集2.0
data = pd.read_csv('./data/Watermelon/Watermelon2.txt', index_col=0)
data = np.array(data)

# 数据编码
oe = OrdinalEncoder(dtype=int)  # 这里注意由于是类别型数据，因为编码时设置为int类型
data_encoded = oe.fit_transform(data)
X_encoded = data_encoded[:, :-1]
y_encoded = data_encoded[:, -1]

In [4]:
# 本文算法
myclf = MyCategoricalNB()
myclf.fit(X_encoded, y_encoded)

X_encoded_test = np.array([[2, 2, 1, 1, 0, 0], [0, 2, 0, 1, 0, 0]])
print('本文算法')
print('-------')
print('预测概率(log)\n', myclf.predict_log_proba(X_encoded_test))
print('预测概率\n', myclf.predict_proba(X_encoded_test))
print('类别对应\n', myclf.classes_)
print('预测类别\n', myclf.predict(X_encoded_test))

本文算法
-------
预测概率(log)
 [[-0.09636785 -2.38737967]
 [-0.03479499 -3.37562894]]
预测概率
 [[0.9081299 0.0918701]
 [0.9658034 0.0341966]]
类别对应
 [1 0]
预测类别
 [1 1]


In [5]:
# sklearn
clf = CategoricalNB()
clf.fit(X_encoded, y_encoded)

print('sklearn')
print('-------')
print('预测概率(log)\n', clf.predict_log_proba(X_encoded_test))
print('预测概率\n', clf.predict_proba(X_encoded_test))
print('类别对应\n', clf.classes_)
print('预测类别\n', clf.predict(X_encoded_test))

sklearn
-------
预测概率(log)
 [[-2.38737967 -0.09636785]
 [-3.37562894 -0.03479499]]
预测概率
 [[0.0918701 0.9081299]
 [0.0341966 0.9658034]]
类别对应
 [0 1]
预测类别
 [1 1]


可以看出，本文算法与sklearn得到的结果一致，只是结果的输出形式有些许差别，这与实际的编码逻辑有关，但不影响

结果的数值。

**<font color=red>说明：</font>**在sklearn 0.24版本中，增加了一个新参数[min_categories](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB)，本文在复现过程中不关注这个参数相关的方法，

重在复现基本的模型，以帮助理解模型的思想和处理过程。

**<font color=black size=4>3 高斯朴素贝叶斯</font>**<br>

高斯朴素贝叶斯假设特征条件独立且服从高斯分布，则：
$$P(X^{(j)}=x^{(j)} | Y=c_k)=\frac{1}{\sqrt{2 \pi \sigma^2_{jk}}}exp(-\frac{(x_i - \mu_{jk})^2}{2 \sigma^2_{jk}})\tag{7}$$
其中：$\sigma^2_{jk}$可以用第$k$类样本中第$j$个特征的方差来估计；$\mu_{jk}$可以用第$k$类样本中第$j$个特征的均值来估计；

**<font color=black size=3.5>3.1 手动实现</font>**<br>

In [6]:
class MyGaussianNB(object):
    """高斯朴素贝叶斯    
    Parameters
    ----------
    priors: ndarray of shape (n_classes,) 
        类别的先验概率，若不指定则根据数据自适应计算
    var_smoothing: float 
        为提高计算的稳定性所设定的方差平滑因子

    Attributes
    ----------
    class_prior_: ndarray of shape (n_classes,) 
        每个类别的概率
    var_smoothing: float 
        方差平滑因子
    classes_: ndarray of shape (n_classes,) 
        已知数据集的分类标签
    var_: ndarray of shape (n_classes, n_features) 
        每种类别的每个特征的方差
    avg_: ndarray of shape (n_classes, n_features) 
        每种类别的每个特征的均值
    """

    def __init__(self, priors=None, var_smoothing=1e-9):
        self.class_prior_ = priors
        self.var_smoothing = var_smoothing
        self.classes_ = None
        self.var_ = None
        self.avg_ = None

    def fit(self, X, y):
        """拟合高斯贝叶斯模型以获得方差、均值等信息
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量
        y:  ndarray of shape (n_samples,) 样本标签
        """
        # 自适应计算类别概率
        c = dict(Counter(y))
        if self.class_prior_ is None:
            self.class_prior_ = np.array(list(c.values())) / len(y)

        # 获取已知数据集的分类标签
        self.classes_ = np.array(list(c.keys()))

        # 计算每种类别的每个特征的方差
        self.var_ = np.array([X[y == label].var(axis=0)
                              for label in self.classes_])
        epsilon = self.var_smoothing * X.var(axis=0).max()
        self.var_ += epsilon  # 添加附加值以调整方差

        # 计算每种类别的每个特征的均值
        self.avg_ = np.array([X[y == label].mean(axis=0)
                              for label in self.classes_])

    def predict(self, X):
        """预测分类标签
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples,) 分类标签
        """
        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_proba(self, X):
        """预测归一化概率
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples, n_classes) 归一化概率
        """
        return np.exp(self.predict_log_proba(X))

    def predict_log_proba(self, X):
        """预测归一化概率(log)
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples, n_classes) 归一化概率(log)
        """
        jll = self._joint_log_likelihood(X)

        # 归一化概率 e.g. log(pi / (p1+p2+...+pn)), i=1,2,...,n, n=n_classes
        log_prob_x = logsumexp(jll, axis=1)
        out = jll - np.atleast_2d(log_prob_x).T
        return out

    def _joint_log_likelihood(self, X):
        """计算联合概率(log)
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        joint_log_likelihood: ndarray of shape (n_samples, n_classes) 联合概率(log)
        """
        joint_log_likelihood = []
        for i in range(len(self.classes_)):
            joint_i = np.log(self.class_prior_[i])
            n_ij = -0.5 * np.sum(np.log(2. * np.pi * self.var_[i, :]))
            n_ij -= 0.5 * np.sum((X-self.avg_[i, :]) ** 2 /
                                 (self.var_[i, :]), 1)
            joint_log_likelihood.append(joint_i + n_ij)

        joint_log_likelihood = np.array(joint_log_likelihood).T
        return joint_log_likelihood

**<font color=black size=3.5>3.2 鸢尾花分类案例</font>**<br>

In [7]:
# 数据准备
iris = load_iris()
X = iris.data
y = iris.target

In [8]:
# 本文算法
myclf2 = MyGaussianNB()
myclf2.fit(X, y)

X_test2 = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print('本文算法')
print('-------')
print('预测概率(log)\n', myclf2.predict_log_proba(X_test2))
print('预测概率\n', myclf2.predict_proba(X_test2))
print('预测类别\n', myclf2.predict(X_test2))

本文算法
-------
预测概率(log)
 [[ -676.68837308   -64.63814755     0.        ]
 [-3008.84065783  -360.48236167     0.        ]]
预测概率
 [[1.31212014e-294 8.47245358e-029 1.00000000e+000]
 [0.00000000e+000 2.78291218e-157 1.00000000e+000]]
预测类别
 [2 2]


In [9]:
# sklearn
clf2 = GaussianNB()
clf2.fit(X, y)

print('sklearn')
print('-------')
print('预测概率(log)\n', clf2.predict_log_proba(X_test2))
print('预测概率\n', clf2.predict_proba(X_test2))
print('预测类别\n', clf2.predict(X_test2))

sklearn
-------
预测概率(log)
 [[ -676.68837308   -64.63814755     0.        ]
 [-3008.84065783  -360.48236167     0.        ]]
预测概率
 [[1.31212014e-294 8.47245358e-029 1.00000000e+000]
 [0.00000000e+000 2.78291218e-157 1.00000000e+000]]
预测类别
 [2 2]


可以看出，本文算法与sklearn得到的结果一致。

**<font size=4 color=black>参考</font>**

1 李航. (2012) 统计学习方法. 清华大学出版社, 北京.

2 [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes)